In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pgeocode
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import LinearSVR
import geopy.distance
!pip3 install pgeocode

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable


In [11]:
def year(date: str) -> int: #возвращает год
    return date[:4]

def season(date: str) -> int: #возвращает сезон года
    m = int(date[4:6])
    season = (m % 12) // 3
    return season


location_pg = pgeocode.Nominatim('US')

def place_of_US(zipcode):
    return location_pg.query_postal_code(str(zipcode))['place_name']


def was_renovated(year): 
    if year != 0:
        return 1
    return 0

def time_renovated(year):
    if year != 0:
        return df['year'] - year
    return df['year'] - df['yr_built']

def coordinates(df):
    x = np.cos(df.lat) * np.cos(df.long)
    y = np.cos(df.lat) * np.sin(df.long) 
    z = np.sin(df.lat) 
    
    return pd.DataFrame({'x': x, 'y': y, 'z': z})

In [12]:
final_list = {'Federal Way': 1,
 'Auburn': 2,
 'Enumclaw': 3,
 'Kent': 4,
 'Maple Valley': 5,
 'Renton': 6,
 'Black Diamond': 7,
 'Duvall': 8,
 'North Bend': 9,
 'Carnation': 10,
 'Kenmore': 11,
 'Seattle': 12,
 'Vashon': 13,
 'Bothell': 14,
 'Fall City': 15,
 'Snoqualmie': 16,
 'Kirkland': 17,
 'Issaquah': 18,
 'Woodinville': 19,
 'Redmond': 20,
 'Sammamish': 21,
 'Bellevue': 22,
 'Mercer Island': 23,
 'Medina': 24}

#создак списан по медиане стоимости дома в каждом городе

In [13]:

def places_by_median_of_price(df):
    list_of_places = df['place'].unique()
    
    places_median = []
    for el in list_of_places:
        tmp_df = df[df['place'] == el]
        places_median.append(np.median(tmp_df['price']))
    print(places_median)
    L = []
    for i in range(len(list_of_places)):
        L.append((places_median[i], list_of_places[i]))
    L.sort()
    final_list = {}
    for i in range(len(list_of_places)):
        final_list[L[i][1]] = i + 1
    return final_list


def column_places_median_f(place):
    return final_list[place]




In [23]:
X = pd.read_csv('/Users/serafim/Desktop/house-pricing-ml-matfac-0/x_train.csv')
y = pd.read_csv('/Users/serafim/Desktop/house-pricing-ml-matfac-0/y_train.csv')
x_test_final = pd.read_csv('/Users/serafim/Desktop/house-pricing-ml-matfac-0/x_test.csv')
dummy = pd.read_csv('/Users/serafim/Desktop/house-pricing-ml-matfac-0/dummy.csv')





In [24]:
# file_places_final = '/Users/serafim/Desktop/Python/машинка/places_by_norm_name_homework_ml.csv'
# file_places_now_X = '/Users/serafim/Desktop/Python/машинка/places_by_norm_homework_ml.csv'


def add_columns_final(df):

    df['year'] = df['date'].apply(year)

    df['year'] = df['year'].astype(int)
    df['old'] = df['year'] - df['yr_built']
    df['sqft_1'] = df['sqft_living'] / (df['floors']+1e-7) / (10**3)
    df['sqft_2'] = df['sqft_living'] / (df['sqft_lot']+1e-7)
    df['sqft_3'] = df['sqft_above'] / (df['sqft_living']+1e-7)
    df['sqft_4'] = df['sqft_basement'] / (df['sqft_living']+1e-7)
    df['sqft_living_per_floor'] = df['sqft_living'] / df['floors']
    df['was_renovated'] = (df['yr_renovated'] != 0)
    df['old_renovated'] = df['year'] - df['yr_renovated']
    df['old_renovated'] = df['old_renovated'].where(df['was_renovated'] == True, df['old'])
    del df['date']
    df = df.join(coordinates(df))
    df['radius'] = (df['x']**2 + df['y']**2)#**(1/2)
    del df['sqft_above']
    del df['sqft_basement']
    del df['sqft_living']
    del df['sqft_lot']
    del df['bedrooms']
    del df['yr_built']
    del df['yr_renovated']   
    df['lat_long'] = df['lat'] - df['long']

    df['place_name'] = df['zipcode'].apply(place_of_US)
#     places_final = pd.read_csv(file_of_places) #скачиваем по ссылке файл с городами
    
#     df['place_name'] = places_final['place']
    df['place_median'] = df['place_name'].apply(column_places_median_f)

    df = pd.get_dummies(df)
    
    poly = PolynomialFeatures(degree = 2)
    df = pd.DataFrame(poly.fit_transform(df), columns = poly.get_feature_names(df.columns))
    del df['1']

    return df

    

    
x_test_final = add_columns_final(x_test_final)
# x_test_final = add_columns_final(x_test_final, file_places_now_X)
X = add_columns_final(X)
    
    
    

/Users/serafim/Library/Python/3.8/lib/python/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/serafim/Library/Python/3.8/lib/python/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [25]:
features_variance = X.var()
features_variance_s = features_variance.sort_values(ascending=False)
# 0.0155
perc = np.percentile(features_variance_s, 43.8)
print(perc)
f_v_s_n = features_variance_s[features_variance_s > 0.0155].index

X = X[f_v_s_n]
x_test_final = x_test_final[f_v_s_n]



0.01570111416723329


In [26]:
# t_0 = 0.006
# E = []
# for i in range(20):
#     # colnames = X.columns


#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)




#     colnames = X.columns
#     scaler_X = StandardScaler()
#     scaler_X.fit(X_train)


#     X_train = scaler_X.transform(X_train)
#     X_test = scaler_X.transform(X_test)
#     # x_test_final = scaler_X.transform(x_test_final)
#     X_train = pd.DataFrame(X_train, columns = colnames)
#     X_test = pd.DataFrame(X_test, columns = colnames)
#     # x_test_final = pd.DataFrame(x_test_final, columns = colnames)
    


#     lin_reg = LinearRegression(n_jobs = -1).fit(X_train, y_train)
#     y_test_pred = lin_reg.predict(X_test)
#     y_train_pred = lin_reg.predict(X_train)
#     E.append((r2_score(y_test_pred, y_test), t_0))
#     t_0 += 0.001
#     features_variance = X.var()
#     features_variance_s = features_variance.sort_values(ascending=False)
#     # 0.0155
#     f_v_s_n = features_variance_s[features_variance_s > t_0].index
#     X = X[f_v_s_n]
#     x_test_final = x_test_final[f_v_s_n]



In [27]:
# plt.plot(E)

In [28]:
# colnames = X.columns


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)




colnames = X.columns
scaler_X = StandardScaler()
scaler_X.fit(X_train)


X_train = scaler_X.transform(X_train)
X_test = scaler_X.transform(X_test)
# x_test_final = scaler_X.transform(x_test_final)
X_train = pd.DataFrame(X_train, columns = colnames)
X_test = pd.DataFrame(X_test, columns = colnames)
# x_test_final = pd.DataFrame(x_test_final, columns = colnames)



lin_reg = LinearRegression(n_jobs=-1).fit(X_train, y_train)
y_test_pred = lin_reg.predict(X_test)
y_train_pred = lin_reg.predict(X_train)
print(f'LinearRegression on test: {r2_score(y_test_pred, y_test)}')
print(f'LinearRegression on train: {r2_score(y_train_pred, y_train)}')
print()
y_test_pred

LinearRegression on test: 0.8901256295052672
LinearRegression on train: 0.9060345835301497



array([[290841.37578991],
       [751253.69170787],
       [521450.47149303],
       ...,
       [643152.2080653 ],
       [595031.77509628],
       [566139.75424677]])

In [29]:
colnames = X.columns
scaler_X = StandardScaler()
scaler_X.fit(X)



X = scaler_X.transform(X)
X = pd.DataFrame(X, columns = colnames)

x_test_final = scaler_X.transform(x_test_final)
x_test_final = pd.DataFrame(x_test_final, columns = colnames)



lin_reg = LinearRegression().fit(X, y)
y_pred = lin_reg.predict(X)
print(f'LinearRegression on: {r2_score(y_pred, y)}')
y_pred
# lin_reg.predict(x_test_final)





LinearRegression on: 0.9038621338073898


array([[733536.15959585],
       [665260.89586294],
       [286205.09093129],
       ...,
       [532513.37633169],
       [191012.23470785],
       [313951.84429276]])

In [30]:
lin_reg.predict(x_test_final)

array([[ 362648.34117544],
       [ 915267.82432973],
       [1246142.26800954],
       ...,
       [ 309676.76817739],
       [ 827055.78795278],
       [ 459181.30890401]])

In [31]:
y_final = pd.DataFrame(np.arange(6484), columns = ['Id'])
y_final['price'] = lin_reg.predict(x_test_final)

y_final
y_final.to_csv('house_pricing_final_version_Serafim.csv')
# y_final.to_csv('/Users/serafim/Desktop/house-pricing-ml-matfac-0/result15.csv', index=False)

,Id,price
0,0,3.626483e+05
1,1,9.152678e+05
2,2,1.246142e+06
3,3,1.932843e+06
4,4,7.113353e+05
...,...,...
6479,6479,3.677616e+05
6480,6480,6.042092e+05
6481,6481,3.096768e+05
6482,6482,8.270558e+05
